# Testing the Neural Network

Import libraries required to test the neural network built

In [3]:
import numpy as np
import main
import random as rd

In this case, we are testing the init_layers function. We are checking if the shape of the weight matrices is correct. If the test passes, we print "All tests pass".
First we try a test case to see if the function is working correctly.

In [4]:
from main import init_layers

def test_init_layers():
    W, b = init_layers(3, [3, 2, 2])
    assert W[0].shape == (2, 3)
    assert W[1].shape == (2, 2)
    assert b[0].shape == (3, 1)
    assert b[1].shape == (2, 1)
    assert b[2].shape == (2, 1)

    print("All tests pass")
    
test_init_layers()

All tests pass


Here it works as expected.
Let's generalize the test case to check if the function is working correctly for all cases.

In [5]:
import random
from main import init_layers

L = random.randint(1, 10)
dims = [random.randint(1, 10) for _ in range(L)]

def test_init_layers():
    W, b = init_layers(L, dims)
    for i in range(L - 1):
        assert W[i].shape == (dims[i + 1], dims[i])
        assert b[i].shape == (dims[i], 1)
    print("All tests pass")
    
test_init_layers()


All tests pass


The function is working correctly for all cases - random number of layers and random number of neurons in each layer.

Now we can test the feedforward function. We will test the function with a simple test case to see if it is working correctly.

In [6]:
from main import feed_forward, sigmoid

def test_forward_propagation():

    W = [np.array([[1.], [2.]]), np.array([[1., 2.]])]
    b = [np.array([[0]]), np.array([[1.], [2.]]), np.array([[3.]])]

    L=3
    # n = [1, 2, 1] number of neurons
    X = np.array([[1.]])
    
    A, Z, y_hat = feed_forward(L, X, W, b)

    assert len(Z) == len(A)
    assert len(A) == L

    # We calculated by hand the expected results, to check if we obtain the matching values

    assert Z[1][0] == 2
    assert A[1][0] == sigmoid(Z[1][0])

    assert Z[1][1] == 4
    assert A[1][1] == sigmoid(Z[1][1])
    
    assert Z[2] == W[1] @ A[1] + b[2]
    assert sigmoid(Z[2]) == y_hat

    """ print("------------------")
    print(W[1] @ A[1] + b[1])
    print(Z[2])
    print("------------------") """
    print("All tests pass")
    
test_forward_propagation()

All tests pass


This use case does work as expected.
Let's now generalize the test to check if the feed forward function works well for any layer and number of neurons per layer.

Then, we'll generalize the test once again to test the robustness of the function based on the dimension of the input matrix X

In [7]:
import random as rd
from main import feed_forward

def test_forward_propagation_n():

    # Limited to 10 layers - 10 layers probably won't ever be need in our case and we don't have infinite calculation power
    L = rd.randint(3, 10)

    dims = []
    for i in range(L):
        dims.append(rd.randint(1, 5))


    # dims = [rd.randint(1, 10) for _ in range(L)] learn generators

    W, b = init_layers(L, dims)
    assert len(W) + 1 == len(b)

    # number of lines of input matrix = n
    # number of columns = 1 for now - robustness test afterwards
    n1 = dims[0]

    # Need to learn the use of generators
    X_list = []
    for i in range(n1):
        X_list.append([rd.randint(1, 200)])

    X = np.array(X_list)

    A, Z, y_hat = feed_forward(L, X, W, b)

    assert len(Z) == len(A)
    assert len(A) == L
    
    for i in range(1, L):
        assert np.array_equal(Z[i], W[i-1] @ A[i-1] + b[i])     
        assert np.array_equal(A[i], sigmoid(Z[i]))

    i = rd.randint(1, L)
    # The values below are the same. As expected.
    # print(W[i-1])
    # print(A[i-1])
    # print(b[i])
    # print(W[i-1] @ A[i-1] + b[i])
    # print("------------")
    # print(Z[i])                     # After calculating by hand, we can see this value is true
    # print("------------")
    # print("------------")
    print(A)
    print(y_hat)
    

    print("------------------")
    print("All tests pass")

test_forward_propagation_n()

[array([[ 58],
       [127],
       [  9],
       [ 69]]), array([[7.99109036e-152],
       [1.18635714e-032],
       [1.58523555e-068]]), array([[0.49765765],
       [0.80063397],
       [0.63623976],
       [0.51462428]]), array([[0.80309738],
       [0.67409184]]), array([[0.05365224],
       [0.15059632],
       [0.51574798]]), array([[0.53989888],
       [0.20044873]]), array([[0.2004548 ],
       [0.78519145]]), array([[0.40221944],
       [0.58620409],
       [0.47826865],
       [0.68699578]]), array([[0.82828535],
       [0.82162712],
       [0.76819194],
       [0.82686906],
       [0.2191913 ]])]
[[0.82828535]
 [0.82162712]
 [0.76819194]
 [0.82686906]
 [0.2191913 ]]
------------------
All tests pass


The function is working correctly for all cases - random number of layers and random number of neurons in each layer.

Now we can test the backpropagation function. We will test the function with a simple test case to see if it is working correctly.

In [11]:
import main
import matplotlib.pyplot as plt


In [9]:
# Initialize layers
nb_layers = 3
dims = [2, 2, 1]
W, b = init_layers(nb_layers, dims)

# Input and target
X = np.array([[1, 2], [3, 4]]).T
y = np.array([[1, 0]])

In [21]:
def plot_gradient_histograms(grad_W, grad_b):
    """
    Plots histograms of the gradients for weights and biases.

    Parameters:
    grad_W: List of weight gradients for each layer.
    grad_b: List of bias gradients for each layer.
    """
    num_layers = len(grad_W)

    plt.figure(figsize=(12, num_layers * 4))
    
    for i in range(num_layers):
        # Plot weight gradients
        plt.subplot(num_layers, 2, i * 2 + 1)
        plt.hist(grad_W[i].flatten(), bins=50, alpha=0.75, color='blue')
        plt.title(f'Layer {i+1} Weight Gradients')
        plt.xlabel('Gradient Value')
        plt.ylabel('Frequency')

        # Plot bias gradients
        plt.subplot(num_layers, 2, i * 2 + 2)
        plt.hist(grad_b[i].flatten(), bins=50, alpha=0.75, color='orange')
        plt.title(f'Layer {i+1} Bias Gradients')
        plt.xlabel('Gradient Value')
        plt.ylabel('Frequency')
        
    plt.savefig("./images/gradient_histograms.png")
    plt.tight_layout()
    plt.show()

# Example usage after backpropagation
A, Z, y_hat = main.feed_forward(nb_layers, X, W, b)
grad_W, grad_b = main.backpropagation(nb_layers, X, y, W, b, A, Z, main.sigmoid_derivative, main.binary_cross_entropy_derivative)





print(grad_W)
print(grad_b)

#plot_gradient_histograms(grad_W, grad_b)



[array([[26.76682227, 35.51070296],
       [16.83195986, 22.36952755]]), array([[-0.8930159 ,  0.01018276]])]
[array([[0.],
       [0.]]), array([[8.74388068],
       [5.53756769]]), array([[-0.89970463]])]
